<a href="https://colab.research.google.com/github/hipatiaj/NLP_course/blob/master/NLP_Course_Week_3_Exercise_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Barajar la data antes de asignar las etiquetas "0 y 1"

In [2]:
!pwd

/content


In [3]:
import json
import tensorflow as tf
import csv
import random
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers


embedding_dim = 100
max_length = 16
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size=160000
#Your dataset size here. Experiment using smaller values (i.e. 16000), but don't forget to train on at least 160000 to see the best effects
test_portion=.1

corpus = []


In [4]:
# Note that I cleaned the Stanford dataset to remove LATIN1 encoding to make it easier for Python CSV reader
# You can do that yourself with:
# iconv -f LATIN1 -t UTF8 training.1600000.processed.noemoticon.csv -o training_cleaned.csv
# I then hosted it on my site to make it easier to use in this notebook

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv \
    -O /tmp/training_cleaned.csv

num_sentences = 0

with open("/tmp/training_cleaned.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    #iterador=iter(reader)
    #lista=list(iterador)
    #random.shuffle(lista)
    for row in reader:
      # Your Code here. Create list items where the first item is the text, found in row[5], and the second is the label. Note that the label is a '0' or a '4' in the text. When it's the former, make
      # your label to be 0, otherwise 1. Keep a count of the number of sentences in num_sentences
        list_item=[]
        # YOUR CODE HERE
        if row[0]=="4":
          row[0]=1
        else:
          row[0]=0
        
        list_item.append(row[5])
        list_item.append(row[0])
        
        num_sentences = num_sentences + 1
        corpus.append(list_item)


--2020-07-24 16:33:44--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.15.128, 173.194.76.128, 2a00:1450:400c:c09::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.15.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238942690 (228M) [application/octet-stream]
Saving to: ‘/tmp/training_cleaned.csv’

/tmp/training_clean 100%[===================>] 227.87M  38.8MB/s    in 5.9s    

2020-07-24 16:33:50 (38.8 MB/s) - ‘/tmp/training_cleaned.csv’ saved [238942690/238942690]



In [6]:
corpus[-20]

['@myheartandmind jo jen by nemuselo zrovna tÃ© holce ael co nic ', 1]

In [ ]:
len(corpus)

1600000

In [ ]:
print(num_sentences)
print(len(corpus))
print(corpus[1])

# Expected Output:
# 1600000
# 1600000
# ["is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!", 0]

1600000
1600000
["is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!", 0]


In [ ]:
sentences=[]
labels=[]
random.shuffle(corpus)
for x in range(training_size):
    sentences.append(corpus[x][0])
    labels.append(corpus[x][1])


tokenizer = Tokenizer(oov_token=oov_tok )
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
vocab_size=len(word_index)

sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences( sequences,maxlen=max_length, padding="post",truncating="post" )

split = int(test_portion * training_size)

test_sequences = padded[0:split]
training_sequences = padded[split:]
test_labels = labels[0:split]
training_labels = labels[split:]

test_labels=np.array(test_labels)
training_labels=np.array(training_labels)

In [ ]:
word_index

{'<OOV>': 1,
 'i': 2,
 'to': 3,
 'the': 4,
 'a': 5,
 'my': 6,
 'and': 7,
 'you': 8,
 'is': 9,
 'it': 10,
 'for': 11,
 'in': 12,
 'of': 13,
 'me': 14,
 'on': 15,
 'so': 16,
 'have': 17,
 'that': 18,
 'but': 19,
 "i'm": 20,
 'just': 21,
 'with': 22,
 'at': 23,
 'be': 24,
 'not': 25,
 'was': 26,
 'this': 27,
 'now': 28,
 'up': 29,
 'good': 30,
 'day': 31,
 'all': 32,
 'out': 33,
 'get': 34,
 'like': 35,
 'are': 36,
 'no': 37,
 'go': 38,
 'http': 39,
 'quot': 40,
 'do': 41,
 'today': 42,
 'too': 43,
 'work': 44,
 "it's": 45,
 'your': 46,
 'going': 47,
 'love': 48,
 'got': 49,
 'lol': 50,
 'time': 51,
 'back': 52,
 'from': 53,
 'u': 54,
 'one': 55,
 'will': 56,
 'com': 57,
 'what': 58,
 'know': 59,
 'about': 60,
 'we': 61,
 'im': 62,
 'really': 63,
 'am': 64,
 "don't": 65,
 'amp': 66,
 'had': 67,
 'can': 68,
 'some': 69,
 'see': 70,
 'its': 71,
 "can't": 72,
 'well': 73,
 'if': 74,
 'still': 75,
 '2': 76,
 'night': 77,
 'new': 78,
 'how': 79,
 'think': 80,
 'want': 81,
 'thanks': 82,
 'home

In [ ]:
print(vocab_size)
print(word_index['i'])
# Expected Output
# 138858
# 1

138359
2


In [ ]:
# Note this is the 100 dimension version of GloVe from Stanford
# I unzipped and hosted it on my site to make this notebook easier
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt


--2020-07-23 23:06:07--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.69.128, 108.177.119.128, 108.177.126.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347116733 (331M) [text/plain]
Saving to: ‘/tmp/glove.6B.100d.txt’

/tmp/glove.6B.100d. 100%[===================>] 331.04M   119MB/s    in 2.8s    

2020-07-23 23:06:10 (119 MB/s) - ‘/tmp/glove.6B.100d.txt’ saved [347116733/347116733]



In [ ]:
embeddings_index = {}
with open('/tmp/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split( )
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;
count=0
embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;
    else:
      count+=1
print(count)

95145


In [ ]:
embeddings_matrix[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
print(len(embeddings_matrix))
# Expected Output
# 138859

138360


In [ ]:
embeddings_matrix.shape

(138360, 100)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1,embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(128,5,activation="relu"),
    tf.keras.layers.Dropout(0.2),
    #tf.keras.layers.Conv1D(128,5,activation="relu"),
   # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalMaxPooling1D(),
    #tf.keras.layers.MaxPooling1D( pool_size=4),
    #tf.keras.layers.Flatten(),
    #
    #
    #tf.keras.layers.Conv1D(64,5,activation="relu"),
    
    #tf.keras.layers.GlobalMaxPooling1D(),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32,activation="relu"),
    
    tf.keras.layers.Dense(1,activation="sigmoid")
])
model.compile(optimizer="adam" ,loss="binary_crossentropy",metrics=["accuracy"])
model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 16, 100)           13836000  
_________________________________________________________________
dropout_8 (Dropout)          (None, 16, 100)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 12, 128)           64128     
_________________________________________________________________
dropout_9 (Dropout)          (None, 12, 128)           0         
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1,embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=True),
    tf.keras.layers.Dropout(0.2),
    #tf.keras.layers.Embedding(vocab_size+1,embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.2),
    #tf.keras.layers.Flatten(),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    
    #tf.keras.layers.Conv1D(128,5,activation="relu"),
    #tf.keras.layers.GlobalMaxPooling1D(),
    
    tf.keras.layers.Dense(32,activation="relu"),
    
    tf.keras.layers.Dense(1,activation="sigmoid")
])
model.compile(optimizer="adam" ,loss="binary_crossentropy",metrics=["accuracy"])
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 16, 100)           13836000  
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 100)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               84480     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 13,924,641
Trainable params: 13,924,641
Non-trainable params: 0
__________________________________________

In [ ]:
num_epochs = 2
history = model.fit(training_sequences, training_labels, epochs=num_epochs, validation_data=(test_sequences, test_labels), verbose=1)

print("Training Complete")


Epoch 1/2
 174/4500 [>.............................] - ETA: 9:03 - loss: 0.6462 - accuracy: 0.6245

KeyboardInterrupt: ignored

## LSTM con dropout


In [ ]:
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Accuracy", "Validation Accuracy"])

plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.title('Training and validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Loss", "Validation Loss"])

plt.figure()


# Expected Output
# A chart where the validation loss does not increase sharply!

## convolucion con dropout

In [ ]:
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Accuracy", "Validation Accuracy"])

plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.title('Training and validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Loss", "Validation Loss"])

plt.figure()


# Expected Output
# A chart where the validation loss does not increase sharply!

## con convolución

In [ ]:
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Accuracy", "Validation Accuracy"])

plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.title('Training and validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Loss", "Validation Loss"])

plt.figure()


# Expected Output
# A chart where the validation loss does not increase sharply!

In [ ]:
tf.random.set_seed(0)
layer = tf.keras.layers.Dropout(.2,noise_shape=(5, 4, 1))
data = np.arange(120).reshape(5,4,6).astype(np.float32)
print(data)

In [ ]:
outputs = layer(data, training=True)
print(outputs)

In [ ]:
import numpy as np
array=np.arange(25).reshape(5,5)

In [ ]:
array[:,:-1]

In [ ]:
array

In [ ]:
import tensorflow as tf
labels=[8,4,5,5,7,3,9,1]
tf.keras.utils.to_categorical(labels, num_classes=15)